# Lora 实战

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "5"

预训练模型中存在一个极小的内在维度，这个内在维度是发挥核心作用的地方。在继续训练的过程中，权重的更新依然也有如此特点，即也存在一个内在维度（内在秩），这就是Lora中低秩矩阵A和B的动机

顺便说一下Adapters微调的核心思想，在多头自注意力机制（Decoder-only结构没有交叉注意力机制，相比Encoder-only结构只是把自注意力机制改成了掩码自注意力机制；Encoder-decoder结构不知道交叉注意力机制是否需要加）和MLP这两个部分的残差链接之前，FFN之后添加一些残差模块（Adapter），包括一个降维矩阵->非线性激活函数->一个升维矩阵和从输入到输出的残差链接，并只优化这些残差模块。其实在结构上与Lora相比就多了非线性激活函数和残差链接！

## Step1 导入相关包

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("../data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [4]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

## Step3 数据集预处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("/data/PLM/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='/data/PLM/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [8]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [9]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

## Step4 创建模型

In [10]:
model = AutoModelForCausalLM.from_pretrained("/data/PLM/bloom-1b4-zh", low_cpu_mem_usage=True)

In [11]:
model # self_attention层的6144是2048*3，包括qkv

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(46145, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (l

In [12]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)

transformer.word_embeddings.weight torch.float32
transformer.word_embeddings_layernorm.weight torch.float32
transformer.word_embeddings_layernorm.bias torch.float32
transformer.h.0.input_layernorm.weight torch.float32
transformer.h.0.input_layernorm.bias torch.float32
transformer.h.0.self_attention.query_key_value.weight torch.float32
transformer.h.0.self_attention.query_key_value.bias torch.float32
transformer.h.0.self_attention.dense.weight torch.float32
transformer.h.0.self_attention.dense.bias torch.float32
transformer.h.0.post_attention_layernorm.weight torch.float32
transformer.h.0.post_attention_layernorm.bias torch.float32
transformer.h.0.mlp.dense_h_to_4h.weight torch.float32
transformer.h.0.mlp.dense_h_to_4h.bias torch.float32
transformer.h.0.mlp.dense_4h_to_h.weight torch.float32
transformer.h.0.mlp.dense_4h_to_h.bias torch.float32
transformer.h.1.input_layernorm.weight torch.float32
transformer.h.1.input_layernorm.bias torch.float32
transformer.h.1.self_attention.query_key_

## Lora

### PEFT Step1 配置文件

In [13]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=["query_key_value"], modules_to_save=["word_embeddings"])
config # target_modules的列表中可以加入其它层，甚至用正则表达式！modules_to_save表示除Lora外仍需训练的层（当然这些层没有Lora）！

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'query_key_value'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=['word_embeddings'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})

### PEFT Step2 创建模型

In [14]:
model = get_peft_model(model, config)

In [15]:
config # 只改变了base_model_name_or_path

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/data/PLM/bloom-1b4-zh', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'query_key_value'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=['word_embeddings'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})

In [16]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): ModulesToSaveWrapper(
          (original_module): Embedding(46145, 2048)
          (modules_to_save): ModuleDict(
            (default): Embedding(46145, 2048)
          )
        )
        (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear(
                in_features=2048, out_features=6144, bias=True
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
           

In [17]:
for name, parameter in model.named_parameters():
    print(name, parameter.dtype)

base_model.model.transformer.word_embeddings.original_module.weight torch.float32
base_model.model.transformer.word_embeddings.modules_to_save.default.weight torch.float32
base_model.model.transformer.word_embeddings_layernorm.weight torch.float32
base_model.model.transformer.word_embeddings_layernorm.bias torch.float32
base_model.model.transformer.h.0.input_layernorm.weight torch.float32
base_model.model.transformer.h.0.input_layernorm.bias torch.float32
base_model.model.transformer.h.0.self_attention.query_key_value.weight torch.float32
base_model.model.transformer.h.0.self_attention.query_key_value.bias torch.float32
base_model.model.transformer.h.0.self_attention.query_key_value.lora_A.default.weight torch.float32
base_model.model.transformer.h.0.self_attention.query_key_value.lora_B.default.weight torch.float32
base_model.model.transformer.h.0.self_attention.dense.weight torch.float32
base_model.model.transformer.h.0.self_attention.dense.bias torch.float32
base_model.model.transfo

In [18]:
model.print_trainable_parameters() # 可训练参数占比这么大完全是word_embeddings的功劳

trainable params: 96,077,824 || all params: 1,399,189,504 || trainable%: 6.866677010178601


## Step5 配置训练参数

In [19]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [20]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Step7 模型训练

In [21]:
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.276300
20,2.929600
30,2.892000
40,2.852600
50,2.702200
60,2.494100
70,2.435400
80,2.497000
90,2.389900
100,2.499000


KeyboardInterrupt: 

## Step8 模型推理

In [22]:
model = model.cuda() # 对哦！这里是直接用训好的model进行测试！
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 在面对考试时，考生需要掌握哪些技巧才能顺利地应付考试呢？\n\n一般来说，有以下一些考试技巧可以帮助您在考试中更好地应对考试：\n\n1. 准备好考试所需的工具：如草稿纸、笔等。考试当天，考生需要为自己准备足够数量的草稿纸和笔，方便在考场上用它们记录和总结答题的要点。\n2. 准备好充足的时间：每个考试科目通常需要考生准备15～20分钟的准备时间，因此在考试前，考生应该尽量做好充足的准备，确保当考试到来的'